In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys
import pymedphys._mocks.profiles
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints


In [ ]:
image_path = pymedphys.data_path('wlutz_image.png')
image_path

In [ ]:
img = imageio.imread(image_path)
assert np.shape(img) == (1024, 1024)
img = img[:, 1:-1]
assert np.shape(img) == (1024, 1022)
assert img.dtype == np.dtype('uint16')
img = 1 - img[::-1,:] / 2**16
assert img.dtype == np.dtype('float64')

In [ ]:
shape = np.shape(img)
x = np.arange(-shape[1]/2, shape[1]/2)/4
y = np.arange(-shape[0]/2, shape[0]/2)/4

In [ ]:
def create_interpolated_field(x, y, img):
    interpolation = scipy.interpolate.RectBivariateSpline(x, y, img.T, kx=1, ky=1)
    
    def field(x, y):
        if np.shape(x) != np.shape(y):
            raise ValueError("x and y required to be the same shape")
            
        result = interpolation.ev(np.ravel(x), np.ravel(y))
        result.shape = x.shape
        
        return result
    
    return field

In [ ]:
field = create_interpolated_field(x, y, img)

In [ ]:
plt.contourf(x, y, img, 30)
plt.axis('equal')
plt.xlim([-25, 25])
plt.ylim([-25, 25])
plt.colorbar()

In [ ]:
xx, yy = np.meshgrid(x, y)

In [ ]:
plt.contourf(x, y, field(xx, yy), 30)
plt.axis('equal')
plt.xlim([-25, 25])
plt.ylim([-25, 25])
plt.colorbar()

In [ ]:
assert np.all(field(xx, yy) == img)